In [2]:
# 共通で利用するライブラリ
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, precision=4)
pd.options.display.float_format = '{:.4f}'.format
pd.set_option("display.max_columns", None)
plt.style.use('seaborn')
plt.rcParams["font.size"] = 14
plt.rcParams['font.family'] = 'IPAexGothic'

# 10章 アンケート分析を行うための言語処理１０本ノック

ここでは、まちづくりのアンケートを使って分析する。  
主に言語処理を取り扱う。
言語処理特有の処理や、データの持たせ方を学び。

- NLP: Natural Language Processing（自然言語処理）

前提条件

- survey.csv
- 「201901~201904」で集めた顧客満足度アンケート
  - アンケート取得日
  - コメント
  - 満足度（5段階）

### Knock91: データを読み込む

In [3]:
# データの読み込み

survey = pd.read_csv("../support/10章/survey.csv")
print(survey.shape)
display(survey)

(86, 3)


,datetime,comment,satisfaction
0,2019/3/11,駅前に若者が集まっている(AA駅),1
1,2019/2/25,スポーツできる場所があるのが良い,5
2,2019/2/18,子育て支援が嬉しい,5
3,2019/4/9,保育園に入れる（待機児童なし）,4
4,2019/1/6,駅前商店街が寂しい,2
...,...,...,...
81,2019/1/21,小学校の校庭が芝生なのでとても良い,5
82,2019/4/30,ホームページからアンケートを投稿できるようにしてほしい,2
83,2019/1/9,公園に遊び道具が少なすぎる,1
84,2019/3/9,もっと公園を増やしてほしい,2


In [5]:
# 欠損値の確認

survey.isna().sum()

datetime        0
comment         2
satisfaction    0
dtype: int64

In [7]:
# 欠損値の除去

survey = survey.dropna()
print(survey.shape)
survey.isna().sum()

(84, 3)


datetime        0
comment         0
satisfaction    0
dtype: int64

### Knock92: 不要な文字を除外する

In [8]:
# 特定文字の除去

survey["comment"] = survey["comment"].str.replace("AA", "")
display(survey)

,datetime,comment,satisfaction
0,2019/3/11,駅前に若者が集まっている(駅),1
1,2019/2/25,スポーツできる場所があるのが良い,5
2,2019/2/18,子育て支援が嬉しい,5
3,2019/4/9,保育園に入れる（待機児童なし）,4
4,2019/1/6,駅前商店街が寂しい,2
...,...,...,...
81,2019/1/21,小学校の校庭が芝生なのでとても良い,5
82,2019/4/30,ホームページからアンケートを投稿できるようにしてほしい,2
83,2019/1/9,公園に遊び道具が少なすぎる,1
84,2019/3/9,もっと公園を増やしてほしい,2


In [9]:
# 正規表現による除去1
# 半角()とカッコ内の除去
survey["comment"] = survey["comment"].str.replace("\(.+?\)", "", regex=True)
display(survey)

,datetime,comment,satisfaction
0,2019/3/11,駅前に若者が集まっている,1
1,2019/2/25,スポーツできる場所があるのが良い,5
2,2019/2/18,子育て支援が嬉しい,5
3,2019/4/9,保育園に入れる（待機児童なし）,4
4,2019/1/6,駅前商店街が寂しい,2
...,...,...,...
81,2019/1/21,小学校の校庭が芝生なのでとても良い,5
82,2019/4/30,ホームページからアンケートを投稿できるようにしてほしい,2
83,2019/1/9,公園に遊び道具が少なすぎる,1
84,2019/3/9,もっと公園を増やしてほしい,2


In [10]:
# 正規表現による除去1
# 全角()とカッコ内の除去
survey["comment"] = survey["comment"].str.replace("\（.+?\）", "", regex=True)
display(survey)

,datetime,comment,satisfaction
0,2019/3/11,駅前に若者が集まっている,1
1,2019/2/25,スポーツできる場所があるのが良い,5
2,2019/2/18,子育て支援が嬉しい,5
3,2019/4/9,保育園に入れる,4
4,2019/1/6,駅前商店街が寂しい,2
...,...,...,...
81,2019/1/21,小学校の校庭が芝生なのでとても良い,5
82,2019/4/30,ホームページからアンケートを投稿できるようにしてほしい,2
83,2019/1/9,公園に遊び道具が少なすぎる,1
84,2019/3/9,もっと公園を増やしてほしい,2


### Knock93: 文字数をカウントしてヒストグラムを表示する